In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import matplotlib.pyplot as plt

import UCI_loader
import importlib

In [28]:
# network class
class Network(nn.Module):
    def __init__(self, indim, outdim):
        super(Network,self).__init__()
        self.l1 = nn.Linear(indim,100)
        self.l2 = nn.Linear(100,50)
        self.l3 = nn.Linear(50,outdim) 
        
    def forward(self,x):
        x = F.leaky_relu(self.l1(x))
        x = F.leaky_relu(self.l2(x))
        x = self.l3(x)
        return x

def tiltedLC(x, y, tau, h):
    e= y-x # errors
    ind= (torch.sign(e)+1)/2 # the division in the log-cosh is only about the origin
    quantFactor= (1-tau)*(1-ind) + tau*ind
    loss= quantFactor*torch.log(torch.cosh(e/h))
    loss= torch.mean(loss)*h
    return loss

def check_loss(x, y, tau): # the x,*args way to pass arguments to this function is an idiom for the scipy.optimize() library y = args[0][0]
    e = y-x
    ind = (torch.sign(-e)+1)/2
    loss = torch.mean(e*(tau-ind))
    return loss

class TiltedLC(nn.Module):
    def __init__(self):
        super(TiltedLC, self).__init__()
    def forward(self, x, y, tau, h):
        return tiltedLC(x, y, tau, h)


class CheckLC(nn.Module):
    def __init__(self):
        super(CheckLC, self).__init__()
    def forward(self, x, y, tau):
        return check_loss(x, y, tau)

# Getting the data
importlib.reload(UCI_loader)
data_energy= UCI_loader.UCIDatasets("energy") 
data_yacht= UCI_loader.UCIDatasets("yacht") 
train_data_energy= data_energy.get_split()
train_data_yacht= data_yacht.get_split()
# test_data= data.get_split(train=False)


# instantiating the network for training on this dataset
device = ('cuda' if torch.cuda.is_available() else 'cpu')
model11= Network(indim=8,outdim=2).to(device=device)
model12= Network(indim=8,outdim=2).to(device=device)
model21= Network(indim=6,outdim=1).to(device=device)
model22= Network(indim=6,outdim=1).to(device=device)
tau= 0.5
h= 0.05

criterion1= TiltedLC()
criterion2= CheckLC()
optimizer11= optim.Adam(model11.parameters())
optimizer12= optim.Adam(model12.parameters())
optimizer21= optim.Adam(model21.parameters())
optimizer22= optim.Adam(model22.parameters())

In [ ]:
# training using the log-cosh
# the fundamental training loop
 # instantiating the DataLoader
trainloader = torch.utils.data.DataLoader(
                    train_data_energy,
                    batch_size=16,
                    shuffle=True)
ls_a= []
N_EPOCHS= 100
for epoch in range(N_EPOCHS):
    epoch_loss1 = 0.0
    for inputs, labels in trainloader:
        inputs = inputs.to(device) 
        labels = labels.to(device)
        optimizer11.zero_grad() 
        outputs = model11(inputs) 
        loss1= criterion1(outputs, labels, tau, h) 
        loss1.backward() 
        optimizer11.step() 
        epoch_loss1+= loss1.item() 
    ls_a.append((epoch_loss1/len(trainloader)))
    print("Epoch: {} Loss: {}".format(epoch,
           epoch_loss1/len(trainloader)))

In [ ]:
# training using the check
ls_b= []
for epoch in range(N_EPOCHS):
    epoch_loss2 = 0.0
    for inputs, labels in trainloader:
        inputs = inputs.to(device) 
        labels = labels.to(device)
        optimizer12.zero_grad() 
        outputs = model12(inputs) 
        loss2= criterion2(outputs, labels, tau)
        loss2.backward() 
        optimizer12.step() 
        epoch_loss2+= loss2.item() 
    ls_b.append(epoch_loss2/len(trainloader))
    print("Epoch: {} Loss: {}".format(epoch,
           epoch_loss2/len(trainloader)))

In [ ]:
plt.plot(range(N_EPOCHS), ls_a) # log-cosh
plt.plot(range(N_EPOCHS), ls_b) # check

In [ ]:
op_1= []
op_2= []

# creating an iterable through each element of the dataset
pltLoad= torch.utils.data.DataLoader(
                    train_data_energy,
                    batch_size=1,
                    shuffle=True)

for inputs, labels in pltLoad:
    inputs = inputs.to(device) 
    labels = labels.to(device)
    outputs1 = model11(inputs) 
    outputs2= model12(inputs)
    op_1.append(outputs1[0][0])
    op_2.append(outputs2[0][0])

plt.plot(range(len(pltLoad)), op_1) # plot of one of the responses for the log-cosh  
plt.plot(range(len(pltLoad)), op_2) # plot of one of the responses for the check